<a href="https://colab.research.google.com/github/kithmini-wijesiri/Antibody-Design-Using-Protein-Language-Models/blob/main/pLM_driven_directed_evolution_using_AbLang.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Antibody Design Using Protein Language Models**


Antibodies are immune proteins also known as immunoglobulins that are naturally produced in the body and bind/inactivate viruses and other pathogens. They are a valuable therapeutic and save lives in immune checkpoint inhibitors for cancer and as neutralizing antibodies for acute viral infection. In addition, they are well known outside the hospital walls for their ability to bind and stick to arbitrary molecular targets, a useful feature in the basic sciences and industrial biochemical facilities.

*Reference:* 2023 Nature Biotechnology paper titled "Efficient evolution of human antibodies from general protein language models" [[1]](https://www.nature.com/articles/s41587-023-01763-2#Sec37) by Hie et al.

## Designing Antibodies via Directed Evolution


### **1. Overview**

![directed_evolution](https://media.springernature.com/full/springer-static/image/art%3A10.1038%2Fs41587-023-01991-6/MediaObjects/41587_2023_1991_Fig1_HTML.png?as=webp)

Image Source: Figure 1. [Outeiral et. al](https://www.nature.com/articles/s41587-023-01991-6)




### **2. Setup/Methodology**


In Hie et al. the authors decide to use a general protein language model instead of one trained specifically on antibody sequences. They use the ESM-1b and ESM-1v models which were trained on UniRef50 and UniRef90 [[2]](https://academic.oup.com/bioinformatics/article/23/10/1282/197795?login=true), respectively. For their directed evolution studies they select seven therapeutic antibodies associated with viral infection spanning Influenza, Ebolavirus, and SARS-COv2. The authors use a straightforward and exhaustive mutation scheduler in mutating every residue in the antigen binding region to every other residue and computing the likelihood of the sequence. Sequences with likelihoods greater than or equal to the WT sequence were kept for experimental validation. Here we take the top-k mutations at a specific point.

Inspired by the work of Hie et al., we first define the pLM driven directed evolution task as simply passing in a masked antibody sequence to a pLM that was previously trained on the masked language modeling objective and examining the token probabilities for the masked amino acids.

For reference we break the task down into the following steps:

```
# Antibody Design via pLM Directed Evolution
1. Select a pre-trained model language model (can be pre-trained on all domains or exclusively antibodies)
2. Choose an antibody to mutate.
3. Determine the amino acid(s) to mask out*.
4. Pass the tokenized sequences into the pLM
5. Sample tokens according to a heuristic for increased fitness
```
*Modification of antibodies needs to focus only on the variable regions as the amino acids at the interface are the ones responsible for driving affinity. Making edits to the constant region would actually be detrimental to antibodies' effector function in the complement system as well as potentially disrupt binding to innate immune receptors. \\




#### 2.1 Loading the Model + Tokenizer

Here we use an early Antibody Language Model AbLang [[3]](https://academic.oup.com/bioinformaticsadvances/article/2/1/vbac046/6609807?login=true). AbLang is a masked language model based on the RoBERTa [[4]](https://arxiv.org/abs/1907.11692) model, and pre-trained on antibody sequences from the [observed antibody space (OAS) [[5]](https://opig.stats.ox.ac.uk/webapps/oas/oas_paired/). AbLang consists of two models, one trained on the heavy chain sequences and one trained on the light chain sequences and the authors demonstrate its usefulness over broader protein language models such as ESM-1b, contradicting the findings put forth in Hie et al. Both the heavy and light chain models are identical in architecture with a $d_{model}$ of 768, max position embedding of 160, and 12 transformer block layers, totaling ~86M parameters.

In [1]:
from transformers import AutoModel, AutoTokenizer, AutoModelForMaskedLM

# Get the tokenizer
tokenizer = AutoTokenizer.from_pretrained('qilowoq/AbLang_light')

# Get the light chain model
mlm_light_chain_model = AutoModelForMaskedLM.from_pretrained('qilowoq/AbLang_light')
# Get the heavy chain model
mlm_heavy_chain_model = AutoModelForMaskedLM.from_pretrained('qilowoq/AbLang_heavy')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/367 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/71.0 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.02k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/848 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/343M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/848 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/343M [00:00<?, ?B/s]

In [2]:
# Lets take a look at the model parameter count and architecture
n_params = sum(p.numel() for p in mlm_heavy_chain_model.parameters())
print(f'The Ablang model has {n_params} trainable parameters. \n')
mlm_heavy_chain_model

The Ablang model has 85809432 trainable parameters. 



RobertaForMaskedLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(24, 768, padding_idx=21)
      (position_embeddings): Embedding(160, 768, padding_idx=21)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): L

#### 2.2 Example Antibody Sequence

Then we choose a heavy chain and light chain for designing. These were chosen from the Ablang examples page on *HuggingFace*.




In [3]:
# Lets take the variable regions of the heavy and light chains

heavy_chain_example =  'EVQLQESGPGLVKPSETLSLTCTVSGGPINNAYWTWIRQPPGKGLEYLGYVYHTGVTNYNPSLKSRLTITIDTSRKQLSLSLKFVTAADSAVYYCAREWAEDGDFGNAFHVWGQGTMVAVSSASTKGPSVFPLAPSSKSTSGGTAALGCL'
light_chain_example = 'GSELTQDPAVSVALGQTVRITCQGDSLRNYYASWYQQKPRQAPVLVFYGKNNRPSGIPDRFSGSSSGNTASLTISGAQAEDEADYYCNSRDSSSNHLVFGGGTKLTVLSQ'

#### 2.2.1 Masking the Sequence

One of the crucial parameters with this approach is in the determination of which residues to mask and re-design. Let's start off by first setting up some reproducible code so that we can apply the masking procedure to any number of sequences at arbitrary points.

In [4]:
# Sequnece masking convenience function
def mask_seq_pos(sequence: str,
                      idx: int,
                      mask='[MASK]'):
    '''Given an arbitrary antibody sequence with and a seqeunce index,
    convert the residue at that index into the mask token.

    '''
    cleaned_sequence = sequence.replace(' ', '')  # Get ride of extraneous spaces if any
    assert abs(idx) < len(sequence), "Zero-indexed value needs to be less than sequence length minus one."
    cleaned_sequence = list(cleaned_sequence)  # Turn the sequence into a list
    cleaned_sequence[idx] = '*'                 # Mask the sequence at idx
    masked_sequence = ' '.join(cleaned_sequence) # Convert list -> seq
    masked_sequence = masked_sequence.replace('*', mask)
    return masked_sequence

# Test
assert mask_seq_pos('CAT', 1)=='C [MASK] T'

#TODO: Add unit tests with pytest where you can check that the assert has been hit

#### 2.3. Model Inference

In [5]:
### Step 1. Mask the light_chain sequence
mask_idx = 9
masked_light_chain = mask_seq_pos(light_chain_example, idx=mask_idx)
### Step 2. Tokenize
tokenized_input = tokenizer(masked_light_chain, return_tensors='pt')
### Step 3. Light Chain Model
mlm_output = mlm_light_chain_model(**tokenized_input)
### Step 4. Decode the outputs to see what the model has placed
decoded_outs = tokenizer.decode(mlm_output.logits.squeeze().argmax(dim=1), skip_special_tokens=True)
print(f'Model predicted: {decoded_outs.replace(" ", "")[9]} at index {mask_idx}')
print(f'Predicted Sequence: {decoded_outs.replace(" ", "")}')
print(f'Starting  Sequence: {light_chain_example}')

Model predicted: S at index 9
Predicted Sequence: SADSSSCGVSSTVAHGQTLKINSQGQRHSLYYVRWYQQKPGLAPLLLIYGKNSRPSGIPDRFSGSKSGTTASLTITGLQAEDEADYYCQQSGGSGGHLTVGGGALLATLTQ
Starting  Sequence: GSELTQDPAVSVALGQTVRITCQGDSLRNYYASWYQQKPRQAPVLVFYGKNNRPSGIPDRFSGSSSGNTASLTISGAQAEDEADYYCNSRDSSSNHLVFGGGTKLTVLSQ


### 2.4 HuggingFace Pipeline Object

Given the tokenized input with only one masked token, we would expect to see only one change to the the sequence. However, what we get back is something a lot more different that what we put in. To address this we can use HuggingFace software suite:

HuggingFace Pipelines:
1. Pipeline object is a wrapper for inference and can be treated like an object for API calls
2. There is a fill-mask pipeline that we can use which accepts a single mask token in out input and outputs a dictionary of the score of that sequence, the imputed token, and the reconstructed full sequence.

In [6]:
from transformers import pipeline
filler = pipeline(task='fill-mask', model=mlm_light_chain_model, tokenizer=tokenizer)
filler(masked_light_chain) # fill in the mask

[{'score': 0.13552618026733398,
  'token': 7,
  'token_str': 'S',
  'sequence': 'G S E L T Q D P A S S V A L G Q T V R I T C Q G D S L R N Y Y A S W Y Q Q K P R Q A P V L V F Y G K N N R P S G I P D R F S G S S S G N T A S L T I S G A Q A E D E A D Y Y C N S R D S S S N H L V F G G G T K L T V L S Q'},
 {'score': 0.11622147262096405,
  'token': 6,
  'token_str': 'E',
  'sequence': 'G S E L T Q D P A E S V A L G Q T V R I T C Q G D S L R N Y Y A S W Y Q Q K P R Q A P V L V F Y G K N N R P S G I P D R F S G S S S G N T A S L T I S G A Q A E D E A D Y Y C N S R D S S S N H L V F G G G T K L T V L S Q'},
 {'score': 0.09913256764411926,
  'token': 9,
  'token_str': 'N',
  'sequence': 'G S E L T Q D P A N S V A L G Q T V R I T C Q G D S L R N Y Y A S W Y Q Q K P R Q A P V L V F Y G K N N R P S G I P D R F S G S S S G N T A S L T I S G A Q A E D E A D Y Y C N S R D S S S N H L V F G G G T K L T V L S Q'},
 {'score': 0.08476438373327255,
  'token': 14,
  'token_str': 'A',
  'sequence': 'G S E 

**We have now designed 5 new antibodies**

Disclaimer: For a more thorough antibody (re)design, we will typically want to follow an approach like what was done in Hie et al. where every point along the sequence will be mutated and the total number of sequences will be collated and scored with the top-100 or so antibodies being expressed for validation.

### **3. Limitations**

* Fixed length antibody design since masked tokens are applied in a 1:1 fashion.
* Lack of target information included during conditional sampling step which can influence choice of amino acid given the sequence context.
* Approach is sensitive to choice of protein language model